In [1]:
#!pip install nibabel
#!pip install nilearn
#!pip install opencv-python
#!pip3 install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118
#!pip install segmentation-models-pytorch
#!pip install imageio
#!pip install matplotlib
#!pip install torchmetrics
#!pip install natsort
!pip install albumentations

In [1]:
import nibabel as nib
import cv2
import numpy as np
import os
import imageio.v2 as imageio
import torch
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
import segmentation_models_pytorch as smp
import torch.optim as optim
import torch.nn as nn
from sklearn.model_selection import train_test_split
import shutil
import matplotlib.pyplot as plt
from torchmetrics.classification import Dice
from natsort import natsorted
import albumentations as A
from skimage.transform import resize

/opt/anaconda/miniconda3/envs/TFG/lib/python3.12/site-packages/albumentations/__init__.py:24: UserWarning: A new version of Albumentations is available: 2.0.3 (you have 1.4.22). Upgrade using: pip install -U albumentations. To disable automatic update checks, set the environment variable NO_ALBUMENTATIONS_UPDATE to 1.
  check_for_updates()


In [2]:
def porcentaje(numero, porcentaje):
    num = numero * (porcentaje/100)
    return int(num)

In [3]:
def obtener_ficheros(carpeta):
    lista_ficheros = []
    for archivo in os.listdir(carpeta):
        ruta = os.path.join(carpeta, archivo)
        if archivo.lower().endswith('.png'):
            lista_ficheros.append(ruta)
    return natsorted(lista_ficheros)

In [4]:
class Custom_Dataset(Dataset):
    def __init__(self, imagenes, mascaras, transform=None, argument=False):
        self.imagenes = imagenes
        self.mascaras = mascaras
        self.transform = transform
        self.argument = argument
        
    def __len__(self):
        return len(self.imagenes)
        
    def __getitem__(self, idx):
        img = imageio.imread(self.imagenes[idx])
        mask = imageio.imread(self.mascaras[idx])

        target_height = 224
        target_width = 224

        img = cv2.resize(img, (target_width, target_height), interpolation = cv2.INTER_AREA)
        mask = cv2.resize(mask, (target_width, target_height), interpolation = cv2.INTER_NEAREST)
        
        img = img/255.0
        mask = mask/255.0
        
        if self.argument:
            transform = A.Compose([
                A.ShiftScaleRotate(shift_limit=0.0625, scale_limit=0.1, rotate_limit=15, p=0.5),
                A.RandomBrightnessContrast(brightness_limit=0.2, contrast_limit=0.2, p=0.5),
                A.CLAHE(clip_limit=2.0, p=0.5)
            ])

            img_for_aug = (img * 255).astype(np.uint8)
            mask_for_aug = (mask * 255).astype(np.uint8)

            augmented = transform(image=img_for_aug, mask=mask_for_aug)

            img = augmented['image'].astype(np.float32) / 255.0
            mask = augmented['mask'].astype(np.float32) / 255.0

        img = np.expand_dims(img, axis=-1)
        mask = np.expand_dims(mask, axis=-1)
        
        img_tensor = torch.from_numpy(img)
        img_tensor = img_tensor.permute(2, 0, 1).float()
        mask_tensor = torch.from_numpy(mask)
        mask_tensor = mask_tensor.permute(2, 0, 1).float()

        return img_tensor, mask_tensor

In [5]:
carpeta = 'Resultados2/Cerebro/Axial/'
lista_ficheros_cerebro_axial = obtener_ficheros(carpeta)
print(len(lista_ficheros_cerebro_axial))

7920


In [6]:
carpeta = 'Resultados2/Mascara/Axial/'
lista_ficheros_mascara_axial = obtener_ficheros(carpeta)
print(len(lista_ficheros_mascara_axial))

7920


In [7]:
carpeta = 'Resultados2/Cerebro/Axial/Val/'
lista_ficheros_cerebro_axial_val = obtener_ficheros(carpeta)
print(len(lista_ficheros_cerebro_axial_val))

1100


In [8]:
carpeta = 'Resultados2/Mascara/Axial/Val/'
lista_ficheros_mascara_axial_val = obtener_ficheros(carpeta)
print(len(lista_ficheros_mascara_axial_val))

1100


In [9]:
train_dataset = Custom_Dataset(lista_ficheros_cerebro_axial,lista_ficheros_mascara_axial, argument=True)
dataloader_train = DataLoader(train_dataset, batch_size=16, shuffle=True)

In [10]:
val_dataset = Custom_Dataset(lista_ficheros_cerebro_axial_val,lista_ficheros_mascara_axial_val)
dataloader_val = DataLoader(val_dataset, batch_size=16, shuffle=False)

In [11]:
model = smp.UnetPlusPlus(
    encoder_name="resnet34",
    encoder_weights="imagenet",
    in_channels=1,
    classes=1
)

In [12]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

cuda


In [13]:
model = model.to(device)
criterion = nn.BCEWithLogitsLoss()
optimizer = optim.Adam(model.parameters(), lr=1e-4)
dice_metric = Dice(average='micro').to(device)

In [14]:
class EarlyStopping():
    def __init__(self, paciencia=15, min_cambio=0.005):

        self.paciencia = paciencia
        self.min_cambio = min_cambio
        self.best_metric = None
        self.contador = 0
        self.early_stop = False

    def __call__(self, dato, model, path='guardado.pth'):
        if self.best_metric is None:
            self.best_metric = dato
            torch.save(model.state_dict(), path)
        elif (dato - self.best_metric) > self.min_cambio:
            self.best_metric = dato
            torch.save(model.state_dict(), path)
            self.contador = 0
        else:
            self.contador += 1
            print(f"Contador earlystopping: {self.contador} sobre {self.paciencia}")
            if self.contador >= self.paciencia:
                self.early_stop = True

In [15]:
perdidas_train = []
perdidas_val = []
dice_train = []
dice_val = []
num_epochs = 200
stop = EarlyStopping(paciencia=15, min_cambio=0.005)

for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    running_dice = 0.0
    
    for images, masks in dataloader_train:
        images = images.to(device)
        masks = masks.to(device)

        outputs = model(images)
        loss = criterion(outputs, masks)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

        predictions = torch.sigmoid(outputs)
        masks = masks.long()
        
        dice = dice_metric(predictions, masks)
        
        running_dice += dice.item()

    perdida = running_loss/len(dataloader_train)
    perdidas_train.append(perdida)

    fiabilidad = running_dice/len(dataloader_train)
    dice_train.append(fiabilidad)
    
    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {perdida}, Dice: {fiabilidad}')

    model.eval()
    val_loss = 0.0
    val_dice = 0.0
    
    with torch.no_grad():
        for images, masks in dataloader_val:
            images = images.to(device)
            masks = masks.to(device)
            
            outputs = model(images)
            loss = criterion(outputs, masks)
            
            val_loss += loss.item()

            predictions = torch.sigmoid(outputs)
            predictions = (predictions > 0.5).float()
            masks = masks.long()
          
            batch_dice = 0.0
            for i in range(images.size(0)):
                if masks[i].sum() == 0 and predictions[i].sum() == 0:
                    dice = torch.tensor(1.0)
                else:
                    dice = dice_metric(predictions[i], masks[i])
    
                batch_dice += dice.item()
            
            batch_dice_media = batch_dice / images.size(0)
            val_dice += batch_dice_media
    
    perdida = val_loss/len(dataloader_val)
    perdidas_val.append(perdida)

    fiabilidad = val_dice/len(dataloader_val)
    dice_val.append(fiabilidad)
    
    print(f'Validation Loss: {perdida}, Validation Dice: {fiabilidad}')

    stop(fiabilidad, model, path='UNet++_mejor_Axial.pth')
    if stop.early_stop:
        print("Early stopping")
        break

Epoch [1/200], Loss: 0.18551275695514197, Dice: 0.05528908502116489
Validation Loss: 0.068498145788908, Validation Dice: 0.5592803226588159
Epoch [2/200], Loss: 0.04271417172493959, Dice: 0.633981221462741
Validation Loss: 0.02447782216620618, Validation Dice: 0.6428214811955034
Epoch [3/200], Loss: 0.018289934570966948, Dice: 0.7180949092513383
Validation Loss: 0.012066071151175362, Validation Dice: 0.658196448409201
Epoch [4/200], Loss: 0.010371507560326294, Dice: 0.7551874790528808
Validation Loss: 0.007738625529505636, Validation Dice: 0.6576557623549782
Contador earlystopping: 1 sobre 15
Epoch [5/200], Loss: 0.00704578421351434, Dice: 0.7689554770787557
Validation Loss: 0.005370266873902385, Validation Dice: 0.6640831911624274
Epoch [6/200], Loss: 0.0053708704197873375, Dice: 0.7831489838735022
Validation Loss: 0.004289888348295421, Validation Dice: 0.6923452363167282
Epoch [7/200], Loss: 0.004368601833479573, Dice: 0.7917932686179575
Validation Loss: 0.0038285741356430926, Valida

Coronal

In [16]:
carpeta = 'Resultados2/Cerebro/Coronal/'
lista_ficheros_cerebro_coronal = obtener_ficheros(carpeta)
print(len(lista_ficheros_cerebro_coronal))

9504


In [17]:
carpeta = 'Resultados2/Mascara/Coronal/'
lista_ficheros_mascara_coronal = obtener_ficheros(carpeta)
print(len(lista_ficheros_mascara_coronal))

9504


In [18]:
carpeta = 'Resultados2/Cerebro/Coronal/Val/'
lista_ficheros_cerebro_coronal_val = obtener_ficheros(carpeta)
print(len(lista_ficheros_cerebro_coronal_val))

1320


In [19]:
carpeta = 'Resultados2/Mascara/Coronal/Val/'
lista_ficheros_mascara_coronal_val = obtener_ficheros(carpeta)
print(len(lista_ficheros_mascara_coronal_val))

1320


In [20]:
train_dataset = Custom_Dataset(lista_ficheros_cerebro_coronal,lista_ficheros_mascara_coronal, argument=True)
dataloader_train = DataLoader(train_dataset, batch_size=16, shuffle=True)

In [21]:
val_dataset = Custom_Dataset(lista_ficheros_cerebro_coronal_val,lista_ficheros_mascara_coronal_val)
dataloader_val = DataLoader(val_dataset, batch_size=16, shuffle=False)

In [22]:
model = smp.UnetPlusPlus(
    encoder_name="resnet34",
    encoder_weights="imagenet",
    in_channels=1,
    classes=1
)

In [23]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

cuda


In [24]:
model = model.to(device)
criterion = nn.BCEWithLogitsLoss()
optimizer = optim.Adam(model.parameters(), lr=1e-4)
dice_metric = Dice(average='micro').to(device)

In [25]:
perdidas_train = []
perdidas_val = []
dice_train = []
dice_val = []
num_epochs = 200
stop = EarlyStopping(paciencia=15, min_cambio=0.005)

for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    running_dice = 0.0
    
    for images, masks in dataloader_train:
        images = images.to(device)
        masks = masks.to(device)

        outputs = model(images)
        loss = criterion(outputs, masks)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

        predictions = torch.sigmoid(outputs)
        masks = masks.long()
        
        dice = dice_metric(predictions, masks)
        
        running_dice += dice.item()

    perdida = running_loss/len(dataloader_train)
    perdidas_train.append(perdida)

    fiabilidad = running_dice/len(dataloader_train)
    dice_train.append(fiabilidad)
    
    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {perdida}, Dice: {fiabilidad}')

    model.eval()
    val_loss = 0.0
    val_dice = 0.0
    
    with torch.no_grad():
        for images, masks in dataloader_val:
            images = images.to(device)
            masks = masks.to(device)
            
            outputs = model(images)
            loss = criterion(outputs, masks)
            
            val_loss += loss.item()

            predictions = torch.sigmoid(outputs)
            predictions = (predictions > 0.5).float()
            masks = masks.long()
          
            batch_dice = 0.0
            for i in range(images.size(0)):
                if masks[i].sum() == 0 and predictions[i].sum() == 0:
                    dice = torch.tensor(1.0)
                else:
                    dice = dice_metric(predictions[i], masks[i])
    
                batch_dice += dice.item()
            
            batch_dice_media = batch_dice / images.size(0)
            val_dice += batch_dice_media
    
    perdida = val_loss/len(dataloader_val)
    perdidas_val.append(perdida)

    fiabilidad = val_dice/len(dataloader_val)
    dice_val.append(fiabilidad)
    
    print(f'Validation Loss: {perdida}, Validation Dice: {fiabilidad}')

    stop(fiabilidad, model, path='UNet++_mejor_Coronal.pth')
    if stop.early_stop:
        print("Early stopping")
        break

Epoch [1/200], Loss: 0.19882993943211608, Dice: 0.4799593657444103
Validation Loss: 0.07637125837156572, Validation Dice: 0.6048868394313739
Epoch [2/200], Loss: 0.04546027647814265, Dice: 0.7289735053223793
Validation Loss: 0.025361827249268453, Validation Dice: 0.6172056285487713
Epoch [3/200], Loss: 0.0178106258582537, Dice: 0.7596769022821176
Validation Loss: 0.012048879694418016, Validation Dice: 0.6375912591500814
Epoch [4/200], Loss: 0.009968816670385324, Dice: 0.7734102873689799
Validation Loss: 0.00732212136667895, Validation Dice: 0.6668506147317499
Epoch [5/200], Loss: 0.006705661007965153, Dice: 0.7880878220704268
Validation Loss: 0.0054982516673359886, Validation Dice: 0.6525403044161285
Contador earlystopping: 1 sobre 15
Epoch [6/200], Loss: 0.005072144009894143, Dice: 0.7953285936153296
Validation Loss: 0.004224569662979029, Validation Dice: 0.6548329461730613
Contador earlystopping: 2 sobre 15
Epoch [7/200], Loss: 0.0040952607860744806, Dice: 0.8046204644421535
Validati

Sagital

In [26]:
carpeta = 'Resultados2/Cerebro/Sagital/'
lista_ficheros_cerebro_sagital = obtener_ficheros(carpeta)
print(len(lista_ficheros_cerebro_sagital))

7920


In [27]:
carpeta = 'Resultados2/Mascara/Sagital/'
lista_ficheros_mascara_sagital = obtener_ficheros(carpeta)
print(len(lista_ficheros_mascara_sagital))

7920


In [28]:
carpeta = 'Resultados2/Cerebro/Sagital/Val/'
lista_ficheros_cerebro_sagital_val = obtener_ficheros(carpeta)
print(len(lista_ficheros_cerebro_sagital_val))

1100


In [29]:
carpeta = 'Resultados2/Mascara/Sagital/Val/'
lista_ficheros_mascara_sagital_val = obtener_ficheros(carpeta)
print(len(lista_ficheros_mascara_sagital_val))

1100


In [30]:
train_dataset = Custom_Dataset(lista_ficheros_cerebro_sagital,lista_ficheros_mascara_sagital, argument=True)
dataloader_train = DataLoader(train_dataset, batch_size=16, shuffle=True)

In [31]:
val_dataset = Custom_Dataset(lista_ficheros_cerebro_sagital_val,lista_ficheros_mascara_sagital_val)
dataloader_val = DataLoader(val_dataset, batch_size=16, shuffle=False)

In [32]:
model = smp.UnetPlusPlus(
    encoder_name="resnet34",
    encoder_weights="imagenet",
    in_channels=1,
    classes=1
)

In [33]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

cuda


In [34]:
model = model.to(device)
criterion = nn.BCEWithLogitsLoss()
optimizer = optim.Adam(model.parameters(), lr=1e-4)
dice_metric = Dice(average='micro').to(device)

In [35]:
perdidas_train = []
perdidas_val = []
dice_train = []
dice_val = []
num_epochs = 200
stop = EarlyStopping(paciencia=15, min_cambio=0.005)

for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    running_dice = 0.0
    
    for images, masks in dataloader_train:
        images = images.to(device)
        masks = masks.to(device)

        outputs = model(images)
        loss = criterion(outputs, masks)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

        predictions = torch.sigmoid(outputs)
        masks = masks.long()
        
        dice = dice_metric(predictions, masks)
        
        running_dice += dice.item()

    perdida = running_loss/len(dataloader_train)
    perdidas_train.append(perdida)

    fiabilidad = running_dice/len(dataloader_train)
    dice_train.append(fiabilidad)
    
    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {perdida}, Dice: {fiabilidad}')

    model.eval()
    val_loss = 0.0
    val_dice = 0.0
    
    with torch.no_grad():
        for images, masks in dataloader_val:
            images = images.to(device)
            masks = masks.to(device)
            
            outputs = model(images)
            loss = criterion(outputs, masks)
            
            val_loss += loss.item()

            predictions = torch.sigmoid(outputs)
            predictions = (predictions > 0.5).float()
            masks = masks.long()
          
            batch_dice = 0.0
            for i in range(images.size(0)):
                if masks[i].sum() == 0 and predictions[i].sum() == 0:
                    dice = torch.tensor(1.0)
                else:
                    dice = dice_metric(predictions[i], masks[i])
    
                batch_dice += dice.item()
            
            batch_dice_media = batch_dice / images.size(0)
            val_dice += batch_dice_media
    
    perdida = val_loss/len(dataloader_val)
    perdidas_val.append(perdida)

    fiabilidad = val_dice/len(dataloader_val)
    dice_val.append(fiabilidad)
    
    print(f'Validation Loss: {perdida}, Validation Dice: {fiabilidad}')

    stop(fiabilidad, model, path='UNet++_mejor_Sagital.pth')
    if stop.early_stop:
        print("Early stopping")
        break

Epoch [1/200], Loss: 0.2599256355955143, Dice: 0.23188754436436504
Validation Loss: 0.09589832254509995, Validation Dice: 0.5185002544962302
Epoch [2/200], Loss: 0.060041269870719526, Dice: 0.618037567265106
Validation Loss: 0.03556923697824064, Validation Dice: 0.5842343607008039
Epoch [3/200], Loss: 0.025851073612769445, Dice: 0.7149953687431836
Validation Loss: 0.017944242238350536, Validation Dice: 0.6369788815154344
Epoch [4/200], Loss: 0.014630949175493284, Dice: 0.7472448227381465
Validation Loss: 0.01128815626050683, Validation Dice: 0.6172968458608108
Contador earlystopping: 1 sobre 15
Epoch [5/200], Loss: 0.009758290012526994, Dice: 0.7670067600529603
Validation Loss: 0.007451651455915492, Validation Dice: 0.6735805277690148
Epoch [6/200], Loss: 0.007166865693800377, Dice: 0.7756671909732048
Validation Loss: 0.0061071344138379545, Validation Dice: 0.6734876984467163
Contador earlystopping: 1 sobre 15
Epoch [7/200], Loss: 0.005807606726790769, Dice: 0.7793158796098497
Validati

Testeo axial

In [5]:
def crear_lista_pacientes_test(carpeta):
    pruebas = []
    for foto in os.listdir(carpeta):
        partes = foto.split('_')
        paciente = '_'.join(partes[:2])
        if paciente not in pruebas:
            pruebas.append(paciente)
    return natsorted(pruebas)

In [6]:
def crear_diccionario_pruebas_test(pruebas):
    return {clave: [] for clave in pruebas}

In [7]:
model = smp.UnetPlusPlus(
    encoder_name="resnet34",
    encoder_weights="imagenet",
    in_channels=1,
    classes=1
)

In [8]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

cuda


In [9]:
model = model.to(device)
dice_metric = Dice(average='micro').to(device)

In [10]:
model.load_state_dict(torch.load("UNet++_mejor_Axial.pth"))

/tmp/ipykernel_124518/1383974660.py:1: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load("UNet++_mejor_Axial.pth"))


<All keys matched successfully>

In [11]:
carpeta = 'Resultados2/Cerebro/Axial/Train/'
lista_ficheros_cerebro_axial_test = obtener_ficheros(carpeta)
print(len(lista_ficheros_cerebro_axial_test))

7920


In [12]:
carpeta = 'Resultados2/Mascara/Axial/Train/'
lista_ficheros_mascara_axial_test = obtener_ficheros(carpeta)
print(len(lista_ficheros_mascara_axial_test))

7920


In [13]:
test_dataset = Custom_Dataset(lista_ficheros_cerebro_axial_test, lista_ficheros_mascara_axial_test)
dataloader_test = DataLoader(test_dataset, batch_size=1, shuffle=False)

In [14]:
carpeta = 'Resultados2/Cerebro/Axial/Train/'
pruebas = crear_lista_pacientes_test(carpeta)
diccionario = crear_diccionario_pruebas_test(pruebas)

dicc_dice = crear_diccionario_pruebas_test(pruebas)

In [15]:
model = model.to(device)
model.eval()

for idx, (images, masks) in enumerate(dataloader_test):
    images = images.to(device)
    masks = masks.to(device)
    
    with torch.no_grad():
        outputs = model(images)
        predictions = torch.sigmoid(outputs)
        predictions = (predictions > 0.5).float()
        masks = masks.long()
        
        if masks.sum() == 0 and predictions.sum() == 0:
            dice = torch.tensor(1.0)
        else:
            dice = dice_metric(predictions, masks)

        dice = dice.item()
    
    original = images.squeeze().cpu().numpy()
    mascara = masks.squeeze().cpu().numpy()
    prediccion = predictions.squeeze().cpu().numpy()

    filename = lista_ficheros_cerebro_axial_test[idx]
    partes = os.path.basename(filename).split('_')
    paciente = '_'.join(partes[:2])

    if paciente in diccionario:
        diccionario[paciente].append(prediccion)
        dicc_dice[paciente].append(dice)
    
for paciente, dice_scores in dicc_dice.items():
    dicc_dice[paciente] = np.mean(dice_scores)

for paciente, dice_score in dicc_dice.items():
    print(f'Paciente: {paciente}, Dice: {dice_score}')

Paciente: P01_T1, Dice: 0.8453110216693445
Paciente: P01_T2, Dice: 0.8766674368896268
Paciente: P01_T3, Dice: 0.8670611959966746
Paciente: P02_T1, Dice: 0.9284899339079857
Paciente: P02_T4, Dice: 0.9036338865756989
Paciente: P03_T2, Dice: 0.8016547436741266
Paciente: P03_T3, Dice: 0.7305170657959852
Paciente: P03_T4, Dice: 0.82818185524507
Paciente: P04_T1, Dice: 0.8744450173594734
Paciente: P04_T2, Dice: 0.7870685119520534
Paciente: P04_T3, Dice: 0.842849757454612
Paciente: P05_T1, Dice: 0.7124398116361012
Paciente: P05_T2, Dice: 0.8177843057973818
Paciente: P06_T1, Dice: 0.7784419444474306
Paciente: P06_T2, Dice: 0.802127819982442
Paciente: P06_T3, Dice: 0.8291117765686729
Paciente: P07_T2, Dice: 0.8522535199468786
Paciente: P08_T2, Dice: 0.788074712997133
Paciente: P09_T2, Dice: 0.7955000178380446
Paciente: P09_T3, Dice: 0.8455659465356307
Paciente: P10_T1, Dice: 0.7757848670536821
Paciente: P10_T2, Dice: 0.7320057616315104
Paciente: P11_T2, Dice: 0.879057497327978
Paciente: P12_T1,

In [16]:
carpeta = 'Datos/Mascara/'
for paciente, imagenes in diccionario.items():
  
    prueba = [f for f in os.listdir(carpeta) if paciente in f][0]
    ruta = os.path.join(carpeta, prueba)
    imagen_nifti = nib.load(ruta)
    datos = imagen_nifti.get_fdata()
    
    num_index = datos.shape[2]
    num_añadir = porcentaje(num_index, 20)

    original_height, original_width = datos.shape[:2]

    imagenes_redimensionadas = np.zeros((num_index, original_height, original_width))


    for idx, img in enumerate(imagenes):
        img_resized = resize(img, (original_height, original_width), mode='constant', preserve_range=True)
        img_resized = (img_resized > 0.5).astype(np.float32)
        imagenes_redimensionadas[idx + num_añadir] = img_resized


    pred_masks_np = np.stack(imagenes_redimensionadas, axis=-1)
    print(pred_masks_np.shape)
    
    nifti_img = nib.Nifti1Image(pred_masks_np, np.eye(4))

    carpeta_nifti = 'Prediccion/Unet++/AxialT'
    if not os.path.exists(carpeta_nifti):
        os.makedirs(carpeta_nifti)
    
    nib.save(nifti_img, os.path.join(carpeta_nifti, f'{paciente}_prediccion_Axial.nii.gz'))

(182, 218, 182)
(182, 218, 182)
(182, 218, 182)
(182, 218, 182)
(182, 218, 182)
(182, 218, 182)
(182, 218, 182)
(182, 218, 182)
(182, 218, 182)
(182, 218, 182)
(182, 218, 182)
(182, 218, 182)
(182, 218, 182)
(182, 218, 182)
(182, 218, 182)
(182, 218, 182)
(182, 218, 182)
(182, 218, 182)
(182, 218, 182)
(182, 218, 182)
(182, 218, 182)
(182, 218, 182)
(182, 218, 182)
(182, 218, 182)
(182, 218, 182)
(182, 218, 182)
(182, 218, 182)
(182, 218, 182)
(182, 218, 182)
(182, 218, 182)
(182, 218, 182)
(182, 218, 182)
(182, 218, 182)
(182, 218, 182)
(182, 218, 182)
(182, 218, 182)
(182, 218, 182)
(182, 218, 182)
(182, 218, 182)
(182, 218, 182)
(182, 218, 182)
(182, 218, 182)
(182, 218, 182)
(182, 218, 182)
(182, 218, 182)
(182, 218, 182)
(182, 218, 182)
(182, 218, 182)
(182, 218, 182)
(182, 218, 182)
(182, 218, 182)
(182, 218, 182)
(182, 218, 182)
(182, 218, 182)
(182, 218, 182)
(182, 218, 182)
(182, 218, 182)
(182, 218, 182)
(182, 218, 182)
(182, 218, 182)
(182, 218, 182)
(182, 218, 182)
(182, 21

Testeo coronal

In [17]:
model = smp.UnetPlusPlus(
    encoder_name="resnet34",
    encoder_weights="imagenet",
    in_channels=1,
    classes=1
)

In [18]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

cuda


In [19]:
model = model.to(device)
dice_metric = Dice(average='micro').to(device)

In [20]:
model.load_state_dict(torch.load("UNet++_mejor_Coronal.pth"))

/tmp/ipykernel_124518/4009313429.py:1: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load("UNet++_mejor_Coronal.pth"))


<All keys matched successfully>

In [21]:
carpeta = 'Resultados2/Cerebro/Coronal/Train/'
lista_ficheros_cerebro_coronal_test = obtener_ficheros(carpeta)
print(len(lista_ficheros_cerebro_coronal_test))

9504


In [22]:
carpeta = 'Resultados2/Mascara/Coronal/Train/'
lista_ficheros_mascara_coronal_test = obtener_ficheros(carpeta)
print(len(lista_ficheros_mascara_coronal_test))

9504


In [23]:
test_dataset = Custom_Dataset(lista_ficheros_cerebro_coronal_test, lista_ficheros_mascara_coronal_test)
dataloader_test = DataLoader(test_dataset, batch_size=1, shuffle=False)

In [24]:
carpeta = 'Resultados2/Cerebro/Coronal/Train/'
pruebas = crear_lista_pacientes_test(carpeta)
diccionario = crear_diccionario_pruebas_test(pruebas)

dicc_dice = crear_diccionario_pruebas_test(pruebas)

In [25]:
model = model.to(device)
model.eval()

for idx, (images, masks) in enumerate(dataloader_test):
    images = images.to(device)
    masks = masks.to(device)
    
    with torch.no_grad():
        outputs = model(images)
        predictions = torch.sigmoid(outputs)
        predictions = (predictions > 0.5).float()
        masks = masks.long()
        
        if masks.sum() == 0 and predictions.sum() == 0:
            dice = torch.tensor(1.0)
        else:
            dice = dice_metric(predictions, masks)

        dice = dice.item()
    
    original = images.squeeze().cpu().numpy()
    mascara = masks.squeeze().cpu().numpy()
    prediccion = predictions.squeeze().cpu().numpy()

    filename = lista_ficheros_cerebro_coronal_test[idx]
    partes = os.path.basename(filename).split('_')
    paciente = '_'.join(partes[:2])

    if paciente in diccionario:
        diccionario[paciente].append(prediccion)
        dicc_dice[paciente].append(dice)
    
for paciente, dice_scores in dicc_dice.items():
    dicc_dice[paciente] = np.mean(dice_scores)

for paciente, dice_score in dicc_dice.items():
    print(f'Paciente: {paciente}, Dice: {dice_score}')

Paciente: P01_T1, Dice: 0.8482138706427632
Paciente: P01_T2, Dice: 0.8653503527695482
Paciente: P01_T3, Dice: 0.9090119473861925
Paciente: P02_T1, Dice: 0.8134804602825281
Paciente: P02_T4, Dice: 0.8290217107895649
Paciente: P03_T2, Dice: 0.7504890299204624
Paciente: P03_T3, Dice: 0.715777959900372
Paciente: P03_T4, Dice: 0.7465936270627108
Paciente: P04_T1, Dice: 0.8709718126690749
Paciente: P04_T2, Dice: 0.8183273284724264
Paciente: P04_T3, Dice: 0.9007620292179512
Paciente: P05_T1, Dice: 0.7572742050344293
Paciente: P05_T2, Dice: 0.7828762685936509
Paciente: P06_T1, Dice: 0.804261901839213
Paciente: P06_T2, Dice: 0.8036163015799089
Paciente: P06_T3, Dice: 0.836940661072731
Paciente: P07_T2, Dice: 0.8898644799535925
Paciente: P08_T2, Dice: 0.80712472624851
Paciente: P09_T2, Dice: 0.8207602875702309
Paciente: P09_T3, Dice: 0.8141266157229742
Paciente: P10_T1, Dice: 0.7382670732384379
Paciente: P10_T2, Dice: 0.7098471264947545
Paciente: P11_T2, Dice: 0.9061558418201677
Paciente: P12_T1

In [26]:
carpeta = 'Datos/Mascara/'
for paciente, imagenes in diccionario.items():

    prueba = [f for f in os.listdir(carpeta) if paciente in f][0]
    ruta = os.path.join(carpeta, prueba)
    imagen_nifti = nib.load(ruta)
    datos = imagen_nifti.get_fdata()

    num_index = datos.shape[1]
    num_añadir = porcentaje(num_index, 20)

    original_depth, original_width = datos.shape[0], datos.shape[2]

    imagenes_redimensionadas = np.zeros((num_index, original_depth, original_width))


    for idx, img in enumerate(imagenes):
        img_resized = resize(img, (original_depth, original_width), mode='constant', preserve_range=True)
        img_resized = (img_resized > 0.5).astype(np.float32)
        imagenes_redimensionadas[idx + num_añadir] = img_resized


    pred_masks_np = np.stack(imagenes_redimensionadas, axis=1)
    print(pred_masks_np.shape)

    nifti_img = nib.Nifti1Image(pred_masks_np, np.eye(4))

    carpeta_nifti = 'Prediccion/Unet++/CoronalT'
    if not os.path.exists(carpeta_nifti):
        os.makedirs(carpeta_nifti)

    nib.save(nifti_img, os.path.join(carpeta_nifti, f'{paciente}_prediccion_Coronal.nii.gz'))

(182, 218, 182)
(182, 218, 182)
(182, 218, 182)
(182, 218, 182)
(182, 218, 182)
(182, 218, 182)
(182, 218, 182)
(182, 218, 182)
(182, 218, 182)
(182, 218, 182)
(182, 218, 182)
(182, 218, 182)
(182, 218, 182)
(182, 218, 182)
(182, 218, 182)
(182, 218, 182)
(182, 218, 182)
(182, 218, 182)
(182, 218, 182)
(182, 218, 182)
(182, 218, 182)
(182, 218, 182)
(182, 218, 182)
(182, 218, 182)
(182, 218, 182)
(182, 218, 182)
(182, 218, 182)
(182, 218, 182)
(182, 218, 182)
(182, 218, 182)
(182, 218, 182)
(182, 218, 182)
(182, 218, 182)
(182, 218, 182)
(182, 218, 182)
(182, 218, 182)
(182, 218, 182)
(182, 218, 182)
(182, 218, 182)
(182, 218, 182)
(182, 218, 182)
(182, 218, 182)
(182, 218, 182)
(182, 218, 182)
(182, 218, 182)
(182, 218, 182)
(182, 218, 182)
(182, 218, 182)
(182, 218, 182)
(182, 218, 182)
(182, 218, 182)
(182, 218, 182)
(182, 218, 182)
(182, 218, 182)
(182, 218, 182)
(182, 218, 182)
(182, 218, 182)
(182, 218, 182)
(182, 218, 182)
(182, 218, 182)
(182, 218, 182)
(182, 218, 182)
(182, 21

Testeo sagital

In [27]:
model = smp.UnetPlusPlus(
    encoder_name="resnet34",
    encoder_weights="imagenet",
    in_channels=1,
    classes=1
)

In [28]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

cuda


In [29]:
model = model.to(device)
dice_metric = Dice(average='micro').to(device)

In [30]:
model.load_state_dict(torch.load("UNet++_mejor_Sagital.pth"))

/tmp/ipykernel_124518/131883476.py:1: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load("UNet++_mejor_Sagital.pth"))


<All keys matched successfully>

In [31]:
carpeta = 'Resultados2/Cerebro/Sagital/Train/'
lista_ficheros_cerebro_sagital_test = obtener_ficheros(carpeta)
print(len(lista_ficheros_cerebro_sagital_test))

7920


In [32]:
carpeta = 'Resultados2/Mascara/Sagital/Train/'
lista_ficheros_mascara_sagital_test = obtener_ficheros(carpeta)
print(len(lista_ficheros_mascara_sagital_test))

7920


In [33]:
test_dataset = Custom_Dataset(lista_ficheros_cerebro_sagital_test, lista_ficheros_mascara_sagital_test)
dataloader_test = DataLoader(test_dataset, batch_size=1, shuffle=False)

In [34]:
carpeta = 'Resultados2/Cerebro/Sagital/Train/'
pruebas = crear_lista_pacientes_test(carpeta)
diccionario = crear_diccionario_pruebas_test(pruebas)

dicc_dice = crear_diccionario_pruebas_test(pruebas)

In [35]:
model = model.to(device)
model.eval()

for idx, (images, masks) in enumerate(dataloader_test):
    images = images.to(device)
    masks = masks.to(device)
    
    with torch.no_grad():
        outputs = model(images)
        predictions = torch.sigmoid(outputs)
        predictions = (predictions > 0.5).float()
        masks = masks.long()
        
        if masks.sum() == 0 and predictions.sum() == 0:
            dice = torch.tensor(1.0)
        else:
            dice = dice_metric(predictions, masks)

        dice = dice.item()
    
    original = images.squeeze().cpu().numpy()
    mascara = masks.squeeze().cpu().numpy()
    prediccion = predictions.squeeze().cpu().numpy()

    filename = lista_ficheros_cerebro_sagital_test[idx]
    partes = os.path.basename(filename).split('_')
    paciente = '_'.join(partes[:2])

    if paciente in diccionario:
        diccionario[paciente].append(prediccion)
        dicc_dice[paciente].append(dice)
    
for paciente, dice_scores in dicc_dice.items():
    dicc_dice[paciente] = np.mean(dice_scores)

for paciente, dice_score in dicc_dice.items():
    print(f'Paciente: {paciente}, Dice: {dice_score}')

Paciente: P01_T1, Dice: 0.9002913036129692
Paciente: P01_T2, Dice: 0.8912588943134655
Paciente: P01_T3, Dice: 0.9261085667393424
Paciente: P02_T1, Dice: 0.889969944412058
Paciente: P02_T4, Dice: 0.847674176571044
Paciente: P03_T2, Dice: 0.8449574141339822
Paciente: P03_T3, Dice: 0.8107828212055292
Paciente: P03_T4, Dice: 0.80605599202893
Paciente: P04_T1, Dice: 0.8727066273039038
Paciente: P04_T2, Dice: 0.8739672007885846
Paciente: P04_T3, Dice: 0.8438642596656626
Paciente: P05_T1, Dice: 0.8451311460950158
Paciente: P05_T2, Dice: 0.8054397653449665
Paciente: P06_T1, Dice: 0.8079658565196124
Paciente: P06_T2, Dice: 0.8386181010441347
Paciente: P06_T3, Dice: 0.8098809104074132
Paciente: P07_T2, Dice: 0.8929244827140461
Paciente: P08_T2, Dice: 0.8555538625202396
Paciente: P09_T2, Dice: 0.8259938868609342
Paciente: P09_T3, Dice: 0.8412802254611795
Paciente: P10_T1, Dice: 0.8210978800600225
Paciente: P10_T2, Dice: 0.812022854252295
Paciente: P11_T2, Dice: 0.8788002174008976
Paciente: P12_T1

In [36]:
carpeta = 'Datos/Mascara/'
for paciente, imagenes in diccionario.items():

    prueba = [f for f in os.listdir(carpeta) if paciente in f][0]
    ruta = os.path.join(carpeta, prueba)
    imagen_nifti = nib.load(ruta)
    datos = imagen_nifti.get_fdata()

    num_index = datos.shape[0]
    num_añadir = porcentaje(num_index, 20)

    original_height, original_depth = datos.shape[1], datos.shape[2]

    imagenes_redimensionadas = np.zeros((num_index, original_height, original_depth))


    for idx, img in enumerate(imagenes):
        img_resized = resize(img, (original_height, original_depth), mode='constant', preserve_range=True)
        img_resized = (img_resized > 0.5).astype(np.float32)
        imagenes_redimensionadas[idx + num_añadir] = img_resized


    pred_masks_np = np.stack(imagenes_redimensionadas, axis=0)
    print(pred_masks_np.shape)

    nifti_img = nib.Nifti1Image(pred_masks_np, np.eye(4))

    carpeta_nifti = 'Prediccion/Unet++/SagitalT'
    if not os.path.exists(carpeta_nifti):
        os.makedirs(carpeta_nifti)

    nib.save(nifti_img, os.path.join(carpeta_nifti, f'{paciente}_prediccion_Sagital.nii.gz'))

(182, 218, 182)
(182, 218, 182)
(182, 218, 182)
(182, 218, 182)
(182, 218, 182)
(182, 218, 182)
(182, 218, 182)
(182, 218, 182)
(182, 218, 182)
(182, 218, 182)
(182, 218, 182)
(182, 218, 182)
(182, 218, 182)
(182, 218, 182)
(182, 218, 182)
(182, 218, 182)
(182, 218, 182)
(182, 218, 182)
(182, 218, 182)
(182, 218, 182)
(182, 218, 182)
(182, 218, 182)
(182, 218, 182)
(182, 218, 182)
(182, 218, 182)
(182, 218, 182)
(182, 218, 182)
(182, 218, 182)
(182, 218, 182)
(182, 218, 182)
(182, 218, 182)
(182, 218, 182)
(182, 218, 182)
(182, 218, 182)
(182, 218, 182)
(182, 218, 182)
(182, 218, 182)
(182, 218, 182)
(182, 218, 182)
(182, 218, 182)
(182, 218, 182)
(182, 218, 182)
(182, 218, 182)
(182, 218, 182)
(182, 218, 182)
(182, 218, 182)
(182, 218, 182)
(182, 218, 182)
(182, 218, 182)
(182, 218, 182)
(182, 218, 182)
(182, 218, 182)
(182, 218, 182)
(182, 218, 182)
(182, 218, 182)
(182, 218, 182)
(182, 218, 182)
(182, 218, 182)
(182, 218, 182)
(182, 218, 182)
(182, 218, 182)
(182, 218, 182)
(182, 21

In [2]:
def copiar_archivos_gz(origen, destino):
    for fichero in os.listdir(origen):
        if fichero.endswith('.gz'):
            ruta_archivo_origen = os.path.join(origen, fichero)
            ruta_archivo_destino = os.path.join(destino, fichero)
            shutil.copy2(ruta_archivo_origen, ruta_archivo_destino)

In [3]:
origen = "Prediccion/Unet++/Axial/"
destino = "DICE_EVAL"
copiar_archivos_gz(origen, destino)

In [37]:
def umbralizacion(input_dir1, input_dir2, input_dir3, output_dir):

    os.makedirs(output_dir, exist_ok=True)

    input_dirs = [input_dir1, input_dir2, input_dir3]
    nifti_files = []
    for dir in input_dirs:
        nifti_files.extend([f for f in os.listdir(dir) if f.endswith(".nii") or f.endswith(".nii.gz")])

    unicas = set("_".join(f.split("_")[:2]) for f in nifti_files)
    
    for cadena in unicas:
        matching_files = [f for f in nifti_files if cadena in f]
        voxel_data_list = []

        for file in matching_files:
            for dir in input_dirs:
                if file in os.listdir(dir):
                    filepath = os.path.join(dir, file)
                    nifti_img = nib.load(filepath)
                    voxel_data = nifti_img.get_fdata()
                    voxel_data_list.append(voxel_data)
                    break

        suma_voxeles = np.sum(voxel_data_list, axis=0)

        umbralizado = (suma_voxeles > 1).astype(np.float32)

        umbralizado_nifti = nib.Nifti1Image(umbralizado, affine=nifti_img.affine, header=nifti_img.header)
        umbralizado_filepath = os.path.join(output_dir, f"{cadena}_prediccion_combinado.nii.gz")
        nib.save(umbralizado_nifti, umbralizado_filepath)

In [38]:
input_dir1 = "Prediccion/Unet++/AxialT/"
input_dir2 = "Prediccion/Unet++/CoronalT/"
input_dir3 = "Prediccion/Unet++/SagitalT/"
output_dir = "Prediccion/Unet++/umbralizacion"

umbralizacion(input_dir1, input_dir2, input_dir3, output_dir)

In [ ]:
origen = "Prediccion/Unet++/nifti_umbralizacion/"
destino = "DICE_EVAL"
copiar_archivos_gz(origen, destino)

In [39]:
def votacion(input_dir1, input_dir2, input_dir3, output_dir):

    os.makedirs(output_dir, exist_ok=True)

    input_dirs = [input_dir1, input_dir2, input_dir3]
    nifti_files = []
    for dir in input_dirs:
        nifti_files.extend([f for f in os.listdir(dir) if f.endswith(".nii") or f.endswith(".nii.gz")])

    unicas = set("_".join(f.split("_")[:2]) for f in nifti_files)
    
    for cadena in unicas:
        matching_files = [f for f in nifti_files if cadena in f]
        voxel_data_list = []

        for file in matching_files:
            for dir in input_dirs:
                if file in os.listdir(dir):
                    filepath = os.path.join(dir, file)
                    nifti_img = nib.load(filepath)
                    voxel_data = nifti_img.get_fdata()
                    voxel_data_list.append((voxel_data > 0).astype(int))
                    break

        suma_voxeles = np.sum(voxel_data_list, axis=0)

        umbralizado = (suma_voxeles >= 2).astype(np.float32)

        umbralizado_nifti = nib.Nifti1Image(umbralizado, affine=nifti_img.affine, header=nifti_img.header)
        umbralizado_filepath = os.path.join(output_dir, f"{cadena}_prediccion_combinado.nii.gz")
        nib.save(umbralizado_nifti, umbralizado_filepath)

In [40]:
input_dir1 = "Prediccion/Unet++/AxialT/"
input_dir2 = "Prediccion/Unet++/CoronalT/"
input_dir3 = "Prediccion/Unet++/SagitalT/"
output_dir = "Prediccion/Unet++/votacion"

votacion(input_dir1, input_dir2, input_dir3, output_dir)

In [41]:
def votacion_ponderada(input_dir1, input_dir2, input_dir3, output_dir, dice_scores):

    os.makedirs(output_dir, exist_ok=True)
    
    input_dirs = [input_dir1, input_dir2, input_dir3]
    nifti_files = []
    for dir in input_dirs:
        nifti_files.extend([f for f in os.listdir(dir) if f.endswith(".nii") or f.endswith(".nii.gz")])
    
    unicas = set("_".join(f.split("_")[:2]) for f in nifti_files)
    
    factor_escala = 3 / sum(dice_scores)
    pesos = [d * factor_escala for d in dice_scores]
    print(pesos)
    for cadena in unicas:
        matching_files = {dir: [f for f in os.listdir(dir) if cadena in f] for dir in input_dirs}
        
        voxel_data_list = []
        
        for i, dir in enumerate(input_dirs):
            for file in matching_files[dir]:
                filepath = os.path.join(dir, file)
                nifti_img = nib.load(filepath)
                voxel_data = nifti_img.get_fdata()
                voxel_data_list.append((voxel_data > 0).astype(float) * pesos[i])
        
        suma_ponderada = np.sum(voxel_data_list, axis=0)
        
        umbralizado = (suma_ponderada >= 1.5).astype(np.float32)
        
        umbralizado_nifti = nib.Nifti1Image(umbralizado, affine=nifti_img.affine, header=nifti_img.header)
        umbralizado_filepath = os.path.join(output_dir, f"{cadena}_prediccion_combinado.nii.gz")
        nib.save(umbralizado_nifti, umbralizado_filepath)

In [42]:
input_dir1 = "Prediccion/Unet++/AxialT/"
input_dir2 = "Prediccion/Unet++/CoronalT/"
input_dir3 = "Prediccion/Unet++/SagitalT/"
output_dir = "Prediccion/Unet++/votacion_ponderada"
dice_scores = [0.761684,0.796344,0.785572]

votacion_ponderada(input_dir1, input_dir2, input_dir3, output_dir, dice_scores)

[0.9750179211469533, 1.0193855606758833, 1.0055965181771633]
